# Mask R-CNN - Nephrology Inference Tool
This is an custom version of [Mask R-CNN - Train cell nucleus Dataset](https://colab.research.google.com/github/navidyou/Mask-RCNN-implementation-for-cell-nucleus-detection-executable-on-google-colab-/blob/master/mask_RCNN_cell_nucleus_google_colab.ipynb) for Google Colab. 

If using this notebook on Google Colab, GPU/TPU might not be used due to version of TensorFlow.

## Google Colab Only

Execute only if using this notebook on Google Colab (installing compatible librairies and getting files needed). Errors might appear, do not worry about this.

In [ ]:
import sys
import os
IN_COLAB = 'google.colab' in sys.modules
print("Executing in Google Colab" if IN_COLAB else "Executing locally")

### Installing compatible librairies

In [ ]:
if IN_COLAB:
    %tensorflow_version 1.x

### Retrieving needed files

You can use this cell to update the files that have been downloaded during the same session and that have been updated on GitHub.

In [ ]:
if IN_COLAB:
    GITHUB_BRANCH = "experimental"
    GITHUB_REPO = f"https://raw.githubusercontent.com/AdrienJaugey/Custom-Mask-R-CNN-for-kidney-s-cell-recognition/{GITHUB_BRANCH}/"
    files = ['mrcnn/config.py', 'mrcnn/utils.py', 'mrcnn/model.py', 'mrcnn/compat.py', 
             'mrcnn/visualize.py', 'mrcnn/post_processing.py', 'datasetTools/datasetDivider.py',
             'datasetTools/datasetWrapper.py', 'datasetTools/AnnotationAdapter.py',
             'datasetTools/ASAPAdapter.py', 'datasetTools/LabelMeAdapter.py', 'nephrology.py', 'common_utils.py']
    for fileToDownload in files:
        url = GITHUB_REPO + fileToDownload
        !wget -qN $url
        if '/' in fileToDownload:
            os.makedirs(os.path.dirname(fileToDownload), exist_ok=True)
            fileName = os.path.basename(fileToDownload)
            !mv $fileName $fileToDownload

### Connecting to Google Drive

The first time this cell is executed, a link should appear, asking you to accept to give access to files of a google account. 
1.   **Follow the link**;
2.   **Choose the account** you want to link;
3.   **Accept**;
4.   **Copy the key** Google gave you;
5.   **Paste the key in the text field** that appeared below the first link you used,
6.   **Press ENTER**.

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import shutil
    shutil.rmtree('sample_data/', ignore_errors=True)

### Retrieving your image(s)

Choose how to get your image(s) from the following list on the right   
Use ```.jp2```, ```.jpg``` or ```.png``` images only !


In [ ]:
if IN_COLAB:
    howToGetImage = "From Google Drive" #@param ["Upload", "From Google Drive"]

In [ ]:
if IN_COLAB:
    !rm -r images/ || true
    !mkdir -p images
    !mkdir -p images/main
    !mkdir -p images/cortex

#### By upload

In [ ]:
if IN_COLAB and howToGetImage == "Upload":
    imageFileType = 'main' #@param ["cortex", "main"]
    print("Please upload the image(s) you want to run the inference on, you can upload the corresponding annotations files too.")
    from google.colab import files
    import shutil
    uploaded = files.upload()
    for fileName in uploaded:
        shutil.move(fileName, "images/{}/{}".format(imageFileType, fileName))

#### By copy from Google Drive

Be sure to customize the 2 variables for Google Colab to be able find your file(s) in Google Drive.
Let's say you have this hierarchy in your Google Drive:
```
Root directory of Google Drive
  ├─── Directory1
  └─── Directory2
       ├─── images
       │    ├─── example1.png
       │    └─── example2.png
       └─── saved_weights
            ├─── main.h5
            └─── cortex.h5
```
1.   ```customPathInDrive``` must represent all the directories between the root directory and your image file. In the example, it would be ```Directory2/images/```. **Do not forget the final /** if you have to use this variable. Keep it empty if **file is directly in root directory** of Google Drive;
2.   ```imageFileName``` must represent the file you want to upload. In the example, it would be ```example1.png```. It can also be empty, if you want to import all the folder's images *(and annotations files if checkbox is checked)* directly to Google Colab, so in the example ```example1.png``` and ```example2.png``` would be imported.

Use the text fields available on the right.

In [ ]:
if IN_COLAB and howToGetImage == "From Google Drive":
    pathToDrive = "'/content/drive/MyDrive/"
    customPathInDrive = "" #@param {type:"string"}
    imageFilePath = "" #@param{type:"string"}
    imageFileType = 'main' #@param ["cortex", "main"]
    annotationsFile = True #@param {type:"boolean"}
    if imageFilePath != "":
        pathToImage = pathToDrive + customPathInDrive + imageFilePath + "'"
        tempPath = "images/" + imageFileType + '/' + imageFilePath
        print("Copying {} to {}".format(pathToImage, tempPath))
        !cp -u $pathToImage $tempPath
        if annotationsFile:
            annotationsFileName = imageFilePath.split('.')[0] + '.xml'
            pathToAnnotations = pathToDrive + customPathInDrive + annotationsFileName + "'"
            tempPath = "images/" + imageFileType + '/' + annotationsFileName
            print("Copying {} to {}".format(pathToAnnotations, tempPath))
            !cp -u $pathToAnnotations $tempPath
    else:
        pathToImageFolder = "/content/drive/MyDrive/" + customPathInDrive
        fileList = os.listdir(pathToImageFolder)
        ext = ['jp2', 'png', 'jpg']
        if annotationsFile:
            ext.extend(['xml', 'json'])
        for dataFile in fileList:
            if dataFile.split('.')[-1] in ext:
                pathToFile = pathToDrive + customPathInDrive + dataFile + "'"
                tempPath = 'images/' + imageFileType + '/' + dataFile
                print("Copying {} to {}".format(pathToFile, tempPath))
                !cp -u $pathToFile $tempPath

### Retrieving Weights File

Same thing than retrieving an image file using Google Drive but it is the saved weights file (```.h5``` extension) for main and cortex modes. With the past example, it would be ```Directory2/saved_weights/``` as ```customPathInDrive```, ```main.h5``` as ```mainWeightFileName``` and ```cortex.h5``` as ```cortexWeightFileName```.

In [ ]:
if IN_COLAB:
    pathToDrive = "'/content/drive/MyDrive/"
    # Keep customPathInDrive empty if file directly in root directory of Google Drive
    customPathInDrive = "" #@param {type:"string"}
    mainWeightFileName = "mask_rcnn_nephrologie_2205_99.h5" #@param {type:"string"}
    cortexWeightFileName = "mask_rcnn_nephrologie_cortex_500_150.h5" #@param {type:"string"}
    for path in [mainWeightFileName, cortexWeightFileName]:
        pathToWeights = pathToDrive + customPathInDrive + path + "'"
        print("Copying {} to {}".format(pathToWeights, path))
        !cp -u $pathToWeights $path

## Initialisation

### Configuration

Be sure to set ```MODEL_PATH``` to the same value than ```weightFileName```. If you want to save the results in files ```saveResults``` should be checked. You will have to open the **Files tab** in the **vertical navigation bar on the left** to see the results appearing. Then you can save them by right-clicking on each file and save it. You can customize the division size but staying at 1024 should be fine. You can also customize the thresholds and the priority table, this will change the post-processing behavior. 

```miniMaskSize``` will reduce memory consumption and duration of the processing (lower is better) however, large elements will be less smooth (reducing accuracy of the mask) (higher is better). Full resolution masks will be used in cortex mode or in main mode only if ```forceFullSizeMasks``` is set to True.

In [ ]:
MAIN_MODEL_PATH = "mask_rcnn_nephrologie_2205_%LAST%.h5" #@param {type:"string"}
CORTEX_MODEL_PATH = "mask_rcnn_nephrologie_cortex_500_%LAST%.h5" #@param {type:"string"}
RESULTS_PATH = "results/inference/" #@param {type:"string"}
MIN_DETECTION_CONFIDENCE = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
DIVISION_SIZE = 1024 #@param {type:"slider", min:896, max:1024, step:1}
miniMaskSize = 128 #@param {type:"slider", min:16, max:256, step:1}
forceFullSizeMasks = False  #@param {type:"boolean"}
MIN_OVERLAP_PART_MAIN = 0.33 #@param {type:"slider", min:0, max:1, step:0.01}
MIN_OVERLAP_PART_CORTEX = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
displayMode = "All steps" #@param ["All steps", "Only AP & statistics"]
saveResults = True #@param {type:"boolean"}
savePreFusionImage = True #@param {type:"boolean"}
savePreFilterImage = True #@param {type:"boolean"}
minMaskArea = 300 #@param {type:"slider", min:0, max:10000000, step:50}
experimental = True #@param {type:"boolean"}
weightId = int(MAIN_MODEL_PATH.split('_')[-2])
cortexMode = False #@param {type:"boolean"}
experimental = experimental and not cortexMode and weightId > 1894
hasNsgClass = weightId > 753
hasIntimaMediaClasses = weightId > 1087
moveFusionDirToImagesAuto = False #@param {type:"boolean"}
cortexSize = (2048, 2048)
MODEL_PATH = CORTEX_MODEL_PATH if cortexMode else MAIN_MODEL_PATH
classesInfo = [
    {"inferenceID": 1,  "id": 0,  "name": "cortex",            "color": "#ffaa00", "ignore": not cortexMode},
    {"inferenceID": 2,  "id": 1,  "name": "medullaire",        "color": "#ff0000", "ignore": not cortexMode},
    {"inferenceID": 3,  "id": 2,  "name": "capsule",           "color": "#ffffff", "ignore": not cortexMode},
    {"inferenceID": 1,  "id": 3,  "name": "tubule_sain",       "color": "#ff007f", "ignore": cortexMode},
    {"inferenceID": 2,  "id": 4,  "name": "tubule_atrophique", "color": "#55557f", "ignore": cortexMode},
    {"inferenceID": 3,  "id": 5,  "name": "nsg_complet",       "color": "#ff557f", "ignore": cortexMode},
    {"inferenceID": 4,  "id": 6,  "name": "nsg_partiel",       "color": "#55aa7f", "ignore": cortexMode},
    {"inferenceID": 5,  "id": 7,  "name": "pac",               "color": "#ffaa7f", "ignore": cortexMode},
    {"inferenceID": 6,  "id": 8,  "name": "vaisseau",          "color": "#55ff7f", "ignore": cortexMode},
    {"inferenceID": 7,  "id": 9,  "name": "artefact",          "color": "#000000", "ignore": cortexMode},
    {"inferenceID": 8,  "id": 10, "name": "veine",             "color": "#0000ff", "ignore": cortexMode},
    {"inferenceID": 9,  "id": 11, "name": "nsg",               "color": "#55007f", "ignore": not hasNsgClass or cortexMode},
    {"inferenceID": 10, "id": 12, "name": "intima",            "color": "#aa0000", "ignore": not hasIntimaMediaClasses or cortexMode},
    {"inferenceID": 11, "id": 13, "name": "media",             "color": "#aa5500", "ignore": not hasIntimaMediaClasses or cortexMode}
]
if weightId > 1894:
    classesInfo = [
        {"inferenceID": 1,  "id": 0,  "name": "cortex",            "color": "#ffaa00", "ignore": not cortexMode},
        {"inferenceID": 2,  "id": 1,  "name": "medullaire",        "color": "#ff0000", "ignore": not cortexMode},
        {"inferenceID": 3,  "id": 2,  "name": "capsule",           "color": "#ffffff", "ignore": not cortexMode},
        {"inferenceID": 1,  "id": 3,  "name": "tubule_sain",       "color": "#ff007f", "ignore": cortexMode},
        {"inferenceID": 2,  "id": 4,  "name": "tubule_atrophique", "color": "#55557f", "ignore": cortexMode},
        {"inferenceID": 3,  "id": 5,  "name": "nsg",               "color": "#55007f", "ignore": cortexMode},
        {"inferenceID": 4,  "id": 6,  "name": "nsg_complet",       "color": "#ff557f", "ignore": cortexMode},
        {"inferenceID": 5,  "id": 7,  "name": "nsg_partiel",       "color": "#55aa7f", "ignore": cortexMode},
        {"inferenceID": 6,  "id": 8,  "name": "pac",               "color": "#ffaa7f", "ignore": cortexMode},
        {"inferenceID": 7,  "id": 9,  "name": "veine",             "color": "#0000ff", "ignore": cortexMode},
        {"inferenceID": 8,  "id": 10, "name": "vaisseau",          "color": "#55ff7f", "ignore": cortexMode},
        {"inferenceID": 9,  "id": 11, "name": "intima",            "color": "#aa0000", "ignore": cortexMode},
        {"inferenceID": 10, "id": 12, "name": "media",             "color": "#aa5500", "ignore": cortexMode}
    ]

# Following variables are used to fuse overlapping masks of the same class
# These thresholds determine the least part represented by the overlapping area
# of bounding boxes or masks to determine whether or not we should fuse two masks
FUSION_BB_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
FUSION_MASK_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

# Following variables are used to filter the masks before displaying results
# Same thing than fusion thresholds but for removing masks
FILTER_BB_THRESHOLD = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
FILTER_MASK_THRESHOLD = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
# Rows and columns correspond to classes in the same order than the classesInfo array without the first element
# An element set to true means that a mask of the column class, if contained by a mask of the
# row class, will be erased.
# True -> colunm element could be erased by row element
if not cortexMode:    #tSain  tAtro  nsg    nsgC   nsgP   pac    veine  vais.  intima media
    PRIORITY_TABLE = [[False, True,  False, False, False, True,  False, False, False, False], # tubule_sain
                      [False, False, False, False, False, True,  False, False, False, False], # tubule_atrophique
                      [True,  True,  False, False, False, True,  True,  True,  True,  True ], # nsg
                      [True,  True,  False, False, True,  True,  True,  True,  False, False], # nsg_complet
                      [True,  True,  False, False, False, True,  True,  True,  False, False], # nsg_partiel
                      [True,  True,  False, False, False, False, False, False, False, False], # pac
                      [False, False, False, False, False, False, False, False, False, False], # veine
                      [True,  True,  False, False, False, True,  True,  False, False, False], # vaisseau
                      [False, False, False, False, False, False, False, False, False, False], # intima
                      [False, False, False, False, False, False, False, False, False, False]] # media
    if experimental:     
        """
        -1 : column element always erase row element
         0 : best element based on score (small mask in big one can "win")
         3 : best element based on score if no inclusion of one in the other 
             Inclusion is supposed when:
             (FILTER_MASK_THRESHOLD * 100)% of mask A < intersection < (FILTER_MASK_THRESHOLD * 100)% of mask B
         1 : row element always erase column element
         2 : do not filter/keep the masks when one is from row class and the other from column class
        """
                         #tS  tAt nsg  nC  nP pac vei vai int med
        PRIORITY_TABLE = [[2,  0,  0,  2,  2,  0,  0,  3,  2,  2], # tubule_sain
                          [0,  2,  0,  2,  2,  0,  0,  3,  2,  2], # tubule_atrophique
                          [0,  0,  2,  2,  2,  0,  1,  3,  2,  2], # nsg
                          [2,  2,  2,  2,  2,  2,  2,  2,  2,  2], # nsg_complet
                          [2,  2,  2,  2,  2,  2,  2,  2,  2,  2], # nsg_partiel
                          [0,  0, -1,  2,  2,  2,  0, -1,  2,  2], # pac
                          [0,  0, -1,  2,  2,  0,  2, -1,  2,  2], # veine
                          [3,  3,  3,  2,  2,  1,  1,  2,  2,  2], # vaisseau
                          [2,  2,  2,  2,  2,  2,  2,  2,  2,  2], # intima
                          [2,  2,  2,  2,  2,  2,  2,  2,  2,  2]] # media
else:                 #cortex medul. capsule
    PRIORITY_TABLE = [[False, False, False], # cortex
                      [False, False, False], # medullaire
                      [False, False, False]] # capsule
# Here are variables 
NB_MAX_DIV_PER_AXIS = 3 #@param {type:"slider", min:3, max:7, step:1}
FUSION_DIV_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

### Information

In [ ]:
print("Skinet in {} segmentation mode :".format("CORTEX" if cortexMode else "MAIN"))
print("\t- Weights file : {}".format(MODEL_PATH))
if cortexMode:
    print("\t- Image(s) will be resized to {} for inference.".format(cortexSize if cortexSize is not None else (1024, 1024)))
    MIN_OVERLAP_PART = MIN_OVERLAP_PART_CORTEX
else :
    print("\t- Masks will be {}".format("full size" if forceFullSizeMasks else "{0}x{0} px".format(miniMaskSize)))
    MIN_OVERLAP_PART = MIN_OVERLAP_PART_MAIN
print("\t- Divisions will be {}x{} px with{} overlapping.".format(DIVISION_SIZE, DIVISION_SIZE, "out" if MIN_OVERLAP_PART == 0 else " {:.2%}".format(MIN_OVERLAP_PART)))

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import nephrology as nephro
images = nephro.listAvailableImage(os.path.join('images', 'cortex' if cortexMode else 'main'))
print("Found {} image{}{}".format(len(images) if len(images) > 0 else 'no', 's' if len(images) > 1 else '', ':' if len(images) > 0 else ''))
for image in images:
    print(" -", image)

## Inference

In [ ]:
import shutil
shutil.rmtree('data/', ignore_errors=True)
if len(images) > 0:
    model = nephro.NephrologyInferenceModel(classesInfo, MODEL_PATH, MIN_DETECTION_CONFIDENCE, DIVISION_SIZE, MIN_OVERLAP_PART_MAIN, MIN_OVERLAP_PART_CORTEX, cortexSize if cortexMode else None, mini_mask_size=miniMaskSize, forceFullSizeMasks=forceFullSizeMasks)
    model.inference(images, RESULTS_PATH, saveResults, FUSION_BB_THRESHOLD, FUSION_MASK_THRESHOLD, FILTER_BB_THRESHOLD, FILTER_MASK_THRESHOLD, PRIORITY_TABLE, NB_MAX_DIV_PER_AXIS, FUSION_DIV_THRESHOLD, displayOnlyAP=(displayMode == "Only AP & statistics"), allowSparse=False, savePreFusionImage=savePreFusionImage, savePreFilterImage=savePreFilterImage, minMaskArea=minMaskArea)

In [ ]:
shutil.rmtree('data/', ignore_errors=True)

### Moving Fusion files (Colab + Local) and downloading results (Colab only)

In [ ]:
if len(images) > 0:
    if RESULTS_PATH is None or RESULTS_PATH in ['.', './', "/", ""]:
        lastDir = "results"
        remainingPath = "./"
    else:
        if RESULTS_PATH[-1] == '/':
            index = -2
        else:
            index = -1
        lastDir = RESULTS_PATH.split('/')[index].replace('/', '')
        remainingPath = RESULTS_PATH[0:RESULTS_PATH.index(lastDir)]

    lastFoundDir = None
    fileList = os.listdir(remainingPath)
    fileList.sort()
    for resultFolder in fileList:
        if lastDir in resultFolder and os.path.isdir(os.path.join(remainingPath, resultFolder)):
            lastFoundDir = resultFolder
    if lastFoundDir is not None:
        lastFoundDirPath = os.path.join(remainingPath, lastFoundDir)
        if cortexMode and moveFusionDirToImagesAuto:
                for imageFolder in os.listdir(lastFoundDirPath):
                    if os.path.isdir(os.path.join(lastFoundDirPath, imageFolder)):
                        fusionFolder = os.path.join(lastFoundDirPath, imageFolder, imageFolder + "_fusion")
                        if os.path.exists(fusionFolder):
                            if len(os.listdir(fusionFolder)) > 2:
                                skinetFile = os.path.join(lastFoundDirPath, imageFolder, imageFolder + "_fusion_info.skinet")
                                shutil.copytree(fusionFolder, os.path.join("images", "main", imageFolder + "_fusion"))
                                shutil.copy2(skinetFile, os.path.join("images", "main", imageFolder + "_fusion_info.skinet"))
        if IN_COLAB:
            zipName = lastFoundDir + '.zip'
            !zip -qr $zipName $lastFoundDirPath
            print("Results can be downloaded on the Files tab on the left")
            print("Zip file name is :", zipName)

This cell may run indefinitely, Google Colab has problem with downloading automatically large files.

In [ ]:
if IN_COLAB and len(images) > 0:
    from google.colab import files
    files.download(zipName)